In [ ]:
!pip install --quiet "pyautogen>=0.2.3"
!pip install -q "pyautogen[retrievechat]"

In [1]:
import autogen

llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "/content/oai_config_list.json",
        filter_dict={"model": ["gpt-4"]},
    ),
    "temperature": 0,
}

In [2]:
from autogen import AssistantAgent, UserProxyAgent, GroupChat, GroupChatManager
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
import sys

In [3]:
llm_config = {
    "timeout": 600,
    "cache_seed": 44,  # change the seed for different trials
    "config_list": autogen.config_list_from_json(
        "/content/oai_config_list.json",
        filter_dict={"model": ["gpt-4"]},
    ),
    "temperature": 0,
}

assistant = RetrieveAssistantAgent(
    name="assistant",
    system_message="You are a helpful assistant.",
    llm_config=llm_config,
)

ragproxyagent = RetrieveUserProxyAgent(
    name="ragproxyagent", #RAG
    retrieve_config={
        "task": "qa",
        "docs_path": "https://www.cs.cmu.edu/~spok/grimmtmp/012.txt",#<Hensel and Gretel> 전문 링크
    },
)

from io import StringIO
import sys
old_stdout = sys.stdout
sys.stdout = mystdout = StringIO()

assistant.reset()
ragproxyagent.reset()
ragproxyagent.initiate_chat(assistant, problem="write a play script of <Hensel and Gretel>, made up of 10 scenes. Each scene should be composed of the narrator's explanation, character's name and the character's line. describe each character's concrete persona with their looks and dressing in brackets, right before each character's line. TERMINATE")

sys.stdout = old_stdout


INFO:autogen.retrieve_utils:Found 2 chunks.


Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: exit


In [19]:
full_script = mystdout.getvalue().split("assistant (to ragproxyagent):\n")[-1].rstrip('\n--------------------------------------------------------------------------------')
#10개 장면으로 만들어진 스크립트 전문

In [21]:
first_character, first_critic_view = "Father", "cold-blooded and uninterested in family"
second_character, second_critic_view = "Stepmother", "Hesitant and indecisive personality"

In [23]:
user_proxy = UserProxyAgent(
    name="user_proxy",
    system_message="""A human admin.""",
    max_consecutive_auto_reply=4,
    llm_config=llm_config,
    human_input_mode="NEVER"
)

player1 = AssistantAgent(
    name="First-editor",
    system_message=f"I think character {first_character} is '{first_critic_view}'. I want to change the play script to fit that. Modifications made by other editors are reflected.",
    llm_config=llm_config,
)

player2 = AssistantAgent(
    name="Second-editor",
    system_message=f"I think character {second_character} is '{second_critic_view}'. I want to change the play script to fit that. Modifications made by other editors are reflected.",
    llm_config=llm_config,
)

groupchat = GroupChat(
    agents=[user_proxy, player1, player2], messages=[]
)
manager = GroupChatManager(groupchat=groupchat, llm_config=llm_config)

#user_proxy.initiate_chat(manager, message="I would love to generate an interesting fairy tale script that showcases the characteristics of each character.")
user_proxy.initiate_chat(manager, message=f"""first, read the script below.
{full_script}

for each scene, revise the story according to your own perspective.
""")

user_proxy (to chat_manager):

first, read the script below.

Scene 1:
Narrator: In a humble cottage near a great forest, a poor wood-cutter, his wife, and their two children, Hansel and Gretel, live a life of scarcity.
Wood-cutter (a man with a weathered face, dressed in worn-out clothes): What is to become of us? How are we to feed our poor children?

Scene 2:
Narrator: Hansel and Gretel overhear their parents' conversation and devise a plan.
Hansel (a young boy with bright eyes, dressed in a small coat): Be quiet, Gretel. I will soon find a way to help us.

Scene 3:
Narrator: The next day, the family ventures into the forest. Hansel leaves a trail of pebbles.
Wood-cutter's wife (a stern woman in a faded dress): Hansel, why are you looking back there and staying behind for?

Scene 4:
Narrator: Left alone in the forest, Hansel and Gretel fall asleep by the fire, only to wake up in the dark.
Gretel (a young girl with tear-streaked face, dressed in a simple frock): How are we to get out

KeyboardInterrupt: 